In [1]:
# LOAD NECESSARY LIBRARIES

import numpy as np
import cv2

def drawRandomLines(NbLines):
    img = np.zeros((800, 800))
    for i in range(NbLines):
        x1 = np.random.randint(low = 0, high=800)
        y1 = np.random.randint(low = 0, high=800)
        x2 = np.random.randint(low = 0, high=800)
        y2 = np.random.randint(low = 0, high=800)
        cv2.line(img, (x1, y1), (x2, y2), (255,255,255), 1)
    img = np.float32(img)
    
    return img

def drawLine(img, mx, rho_max, m, n):

    theta = np.radians(mx[0])
    rho = mx[1] - rho_max

    c = np.cos(theta)
    s = np.sin(theta)
    
    if np.isclose(s, 0):
        pt1 = (0, rho)
        pt2 = (n, rho)

    else:
        pt1 = (int(rho // s), 0)
        pt2 = (int(rho // s - c / s * m), m)

    cv2.line(img, pt1, pt2, (255,0,255), 3)

def show(I, title):
    I = np.uint8(I / np.max(I) * 255)
    cv2.imshow(title, I)
    cv2.waitKey()
    cv2.destroyAllWindows()



In [6]:
def initIndices(edges):
    args = np.argwhere(edges > 0)
    J = list(args[:, 1])
    I = list(args[:, 0])
    return I, J

In [7]:
# RANDOM SAMPLE FROM INDICES

def randomSample(I, J):
    N = len(I)
    i = np.random.randint(0, N)
    x, y = I.pop(i), J.pop(i)
    return (x, y)

In [8]:
# PERFORM HOUGH TRANSFORM

def constructRho(pt):
    rho = np.round(pt[0] * COS_TABLE + pt[1] * SIN_TABLE + RHO_MAX)
    return np.array(rho, dtype=np.int64)

In [9]:
def incrementAccum(accum, maxs):

    pt1 = randomSample(I, J)
    pt2 = randomSample(I, J)
    
    rho1 = constructRho(pt1)
    rho2 = constructRho(pt2)

    max_val = np.max(accum)
    max_val = max_val if max_val > THRESHOLD else THRESHOLD

    accum[_, rho1] += 1
    accum[_, rho2] += 1

    val = np.max(accum)
    
    if val > max_val:
        res = np.argwhere(accum == val)[0]
        theta, rho = res
        
        if (theta, rho) not in maxs:
            maxs.append((theta, rho))

        accum = np.where(accum == val, 0, accum)
        
    return maxs
   

In [10]:
def randomHoughLines():
    maxs = []
    i = 0
    accum = np.zeros((180, int(2 * RHO_MAX)))

    while(i < NUM_OF_ITERATIONS and len(I) > 1):
        maxs = incrementAccum(accum, maxs)

    return maxs

In [13]:
# INIT ALL CONSTANTS

_ = np.arange(0, 180, 1)
COS_TABLE = np.cos(np.radians(_))
SIN_TABLE = np.sin(np.radians(_))

IMAGE_URL = 'tst1.png'
   
IMAGE = 255 - cv2.imread(IMAGE_URL, 0)
EDGES = cv2.Canny(IMAGE, 80, 150)

HEIGHT, WIDTH = EDGES.shape
RHO_MAX = np.hypot(HEIGHT, WIDTH)

THRESHOLD = 10
NUM_OF_ITERATIONS = int(HEIGHT * WIDTH * 0.3)

I, J = initIndices(EDGES)

In [14]:
mxs = randomHoughLines()

show(IMAGE, 'Original')

for mx in mxs:
    print(mx)
    drawLine(IMAGE, mx = mx, rho_max = RHO_MAX, m = HEIGHT, n = WIDTH)

show(IMAGE, 'Detected')

(146, 760)
(147, 704)
(147, 703)
(146, 757)
